In [ ]:
%%capture
!pip install bertviz

In [ ]:
%%capture
#@title Installs
!pip install einops
!pip install torchvision
# !pip install transformers==4.26.1
!pip install genomic-benchmarks
!pip install OmegaConf
!FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE pip install flash-attn --no-build-isolation
!pip install torch --index-url https://download.pytorch.org/whl/cu118
!pip install transformers[torch] datasets

In [ ]:
from transformers import AutoTokenizer, AutoModel, BertForSequenceClassification, AutoModelForMaskedLM, AutoModelForSequenceClassification
import importlib
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import Dataset
from torch.utils.data import DataLoader

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, matthews_corrcoef
import numpy as np
from scipy.special import softmax
import pandas as pd
from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


DNABERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("powidla/DNABERT_pre", trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained("powidla/DNABERT_pre", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/359M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at powidla/DNABERT_pre and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GENA-LM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('AIRI-Institute/gena-lm-bert-base-t2t')
model = AutoModel.from_pretrained('AIRI-Institute/gena-lm-bert-base-t2t', trust_remote_code=True)
gena_module_name = model.__class__.__module__
cls = getattr(importlib.import_module(gena_module_name), 'BertForSequenceClassification')
model = cls.from_pretrained('AIRI-Institute/gena-lm-bert-base-t2t', num_labels=2, output_attentions=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


[2024-08-26 12:31:52,866] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/541M [00:00<?, ?B/s]

Some weights of the model checkpoint at AIRI-Institute/gena-lm-bert-base-t2t were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AIRI-Ins

In [ ]:
!pip install deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
  Using cached triton-2.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.3 MB/s eta 0:00:00
Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.1 MB/s eta 0:00:00
Using cached triton-2.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (168.1 MB)
  Created wheel for deepspeed: filename=deepspeed-0.15.0-py3-none-any.whl size=1483700 sha256=4ad27aa83989c4d56ac0792b4dfd4f0b11bdbfbc02de5502151bf3c1f89788a9
  Stored in directory: /root/.cache/pip/wheels/46/5a/dd/2f36986baec22867dab84e6176db30353ef54e7415cce5f2e0
Successfully built deepspeed


DNABERT2

In [ ]:
!pip install einops transformers==4.27 peft omegaconf torch evaluate accelerate numpy scikit-learn Pillow textaugment scipy
!pip uninstall triton

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.2 MB/s eta 0:00:00
  Created wheel 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


...

bert_padding.py:   0%|          | 0.00/6.10k [00:00<?, ?B/s]

flash_attn_triton.py:   0%|          | 0.00/42.7k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/468M [00:00<?, ?B/s]

/root/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification 

In [ ]:
class HG38Dataset(torch.utils.data.Dataset):

###your code goes here

In [ ]:
num_epochs = 20
max_length = 512
path = "/content/g4_dataset.csv"
batch_size = 16
learning_rate = 8e-03
weight_decay = 0.0
betas=(0.95, 0.9)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

###your code goes here

Using device: cuda


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (pre_attention_ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (post_attention_ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              

In [ ]:
sum(p.numel() for p in model.parameters())

110618882

In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for seqs, targets in train_loader:
        seqs, targets = seqs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(seqs)
        logits = outputs.logits
        loss = loss_fn(logits, targets.squeeze())  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Optimizer step


        train_loss += loss.item()

        _, predicted = torch.max(logits, 1)
        correct_train += (predicted == targets.squeeze()).sum().item()
        total_train += targets.size(0)

    train_loss /= len(train_loader)
    train_accuracy = correct_train / total_train

    model.eval()
    eval_loss = 0.0
    correct_eval = 0
    total_eval = 0

    with torch.no_grad():
        for seqs, targets in test_loader:
            seqs, targets = seqs.to(device), targets.to(device)

            outputs = model(seqs)
            logits = outputs.logits
            loss = loss_fn(logits, targets.squeeze())

            eval_loss += loss.item()

            _, predicted = torch.max(logits, 1)
            correct_eval += (predicted == targets.squeeze()).sum().item()
            total_eval += targets.size(0)

    eval_loss /= len(test_loader)
    eval_accuracy = correct_eval / total_eval


    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Eval Loss: {eval_loss:.4f}, Eval Accuracy: {eval_accuracy:.4f}")

In [ ]:
ds_test = HG38Dataset(
    split = 'test',
    path = path,
    max_length = max_length,
    tokenizer=tokenizer
)

test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=False, drop_last=True)

outputs = []
targets = []

model.eval()
###your code goes here

<ipython-input-3-7b7bf90f636d>:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  seq, target = (row[0], row[1])


In [ ]:
def compute_metrics(outputs, targets):
    predictions = np.argmax(outputs, axis=-1)
    scores = softmax(outputs, axis=-1)[:, 1]
    r = {
        'accuracy': accuracy_score(targets, predictions),
        'f1_score': f1_score(targets, predictions),
        'ROC_AUC': roc_auc_score(targets, scores),
        'MCC': matthews_corrcoef(targets, predictions)
    }
    return r

In [ ]:
metrics = compute_metrics(outputs, targets)

print("DNABERT")
print(f"Accuracy on the test dataset: %0.4f" % metrics['accuracy'])
print(f"F1-score on the test dataset: %0.4f" % metrics['f1_score'])
print(f"ROC AUC on the test dataset: %0.4f" % metrics['ROC_AUC'])
print(f"MCC on the test dataset: %0.4f" % metrics['MCC'])

Attention visualization

In [ ]:
from bertviz import head_view

tokenizer = AutoTokenizer.from_pretrained('AIRI-Institute/gena-lm-bert-base-t2t')
sentence = "TCGGAACCGTGCTTGCAGCCCCCGGCTGGCCGGGGGTCGGGGGCACGCGGATATGTGGGGAAATAAAGGACACTGGAAAAACCGAAACTCTCAAGAGGCT"
inputs = tokenizer.encode(sentence, return_tensors='pt', add_special_tokens=False)
outputs = model(inputs.to(device))
attention = outputs[-1]
tokens = tokenizer.convert_ids_to_tokens(inputs[0])

In [ ]:
head_view(attention, tokens)

<IPython.core.display.Javascript object>

In [ ]:
html_head_view = head_view(attention, tokens, html_action='return')
with open("attention_GENA.html", 'w') as file:
    file.write(html_head_view.data)